# 2-Arm ANCOVA Model for Continuous Outcomes (No Intercept, Fixed Error Variance)

## Model Specification

This notebook implements a 2-arm ANCOVA model **without an intercept** and with **fixed error variance**:

$$\text{outcome} = b_1 \cdot \text{covariate} + b_2 \cdot \text{group} + \epsilon$$

where:
- $b_1$: coefficient for the baseline covariate **(NOT estimated - fixed in simulation)**
- $b_2$: treatment effect (difference between groups) **(parameter to estimate via BayesFlow)**
- $\text{group}$: binary indicator (0 = control, 1 = treatment)
- $\epsilon \sim N(0, 1)$: residual error with **FIXED variance = 1**

**Key features:**
- No intercept term ($\beta_0 = 0$)
- Only 1 parameter estimated: $b_2$ (treatment effect)
- $b_1$ (covariate effect) is simulated but not inferred
- Error variance $\sigma^2 = 1$ (not estimated)

In [ ]:
import os

if not os.environ.get("KERAS_BACKEND"):
    os.environ["KERAS_BACKEND"] = "torch"
    
from pathlib import Path
import importlib

import numpy as np
np.set_printoptions(suppress=True)
RNG = np.random.default_rng(2025)

from itertools import product

import keras
import bayesflow as bf

# plotting
import matplotlib.pyplot as plt

# Import ANCOVA-specific functions and configs from the package
from rctbp_bf_training.models.ancova.model import (
    ANCOVAConfig,
    PriorConfig,
    MetaConfig,
    NetworkConfig,
    TrainingConfig,
    create_adapter,
    create_simulator,
    build_networks,
    create_validation_grid,
    make_simulate_fn,
    make_infer_fn,
    simulate_cond_batch,
    get_model_metadata,
    save_model_with_metadata,
    load_model_with_metadata,
)
from rctbp_bf_training.core.utils import MovingAverageEarlyStopping, loguniform_int

# Create default configuration
config = ANCOVAConfig()
print(f"Config loaded. Model type: ancova_cont_2arms")

## Simulator Components

The simulator is built from three functions (defined in `ancova_cont_2arms_fns.py`):
- **prior**: Samples `b_covariate` and `b_group` parameters
- **likelihood**: Simulates ANCOVA data given parameters
- **meta**: Samples context variables (N, p_alloc, prior_df, prior_scale)

In [ ]:
# Prior, likelihood, meta functions are all in ancova_cont_2arms_fns
# We use create_simulator() to build the complete simulator with config injection
print("Simulator functions defined in ancova_cont_2arms_fns:")
print("  - prior(prior_df, prior_scale, config, rng)")
print("  - likelihood(b_covariate, b_group, N, p_alloc, rng)")
print("  - meta(config, rng)")

In [ ]:
# Create simulator using factory function (injects config and RNG)
simulator = create_simulator(config, RNG)

# Test sampling
sim_draws = simulator.sample(100)
print("Simulator output keys:", sim_draws.keys())
print(f"N: {sim_draws['N']}")
print(f"p_alloc: {sim_draws['p_alloc']:.3f}")
print(f"prior_df: {sim_draws['prior_df']}")
print(f"prior_scale: {sim_draws['prior_scale']:.3f}")
print(f"b_covariate shape: {sim_draws['b_covariate'].shape}")
print(f"b_group shape: {sim_draws['b_group'].shape}")
print(f"outcome[0] shape: {sim_draws['outcome'][0].shape}")

In [ ]:
# Visualize the Gamma distribution for prior_scale
from scipy import stats

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Gamma(shape=2, scale=1) for prior_scale
x_gamma = np.linspace(0, 8, 200)
shape, scale = 2, 1
gamma_pdf = stats.gamma.pdf(x_gamma, a=shape, scale=scale)

ax = axes[0]
ax.plot(x_gamma, gamma_pdf, 'b-', lw=2)
ax.fill_between(x_gamma, gamma_pdf, alpha=0.3)
ax.axvline(shape * scale, color='r', ls='--', label=f'Mean = {shape * scale}')
ax.axvline((shape - 1) * scale, color='g', ls='--', label=f'Mode = {(shape - 1) * scale}')
ax.set_xlabel('prior_scale')
ax.set_ylabel('Density')
ax.set_title(f'Gamma(shape={shape}, scale={scale}) for prior_scale')
ax.legend()
ax.set_xlim(0, 8)

# Log-uniform distribution for prior_df (0 to 30)
ax = axes[1]
# Sample from the actual distribution
samples_df = [int(round(loguniform_int(1, 31, alpha=0.7) - 1)) for _ in range(10000)]
ax.hist(samples_df, bins=np.arange(-0.5, 31.5, 1), density=True, alpha=0.7, edgecolor='black')
ax.set_xlabel('prior_df')
ax.set_ylabel('Density')
ax.set_title('Log-uniform(1, 31, α=0.7) - 1 for prior_df')
ax.set_xlim(-1, 31)

plt.tight_layout()
plt.show()

print(f"prior_scale: Gamma(shape=2, scale=1) → mean={shape*scale}, mode={(shape-1)*scale}")
print(f"prior_df: integer in [0, 30], log-uniform with α=0.7 (skewed toward lower values)")

## Adapter

Transform simulator output for neural networks.

In [ ]:
# Adapter created via factory function
adapter = create_adapter()
    
# Test the adapter
processed = adapter(sim_draws)
print("Processed data shapes:")
print("  inference_variables:", processed["inference_variables"].shape)
print("  inference_conditions:", processed["inference_conditions"].shape)
print("  summary_variables:", processed["summary_variables"].shape)

## Networks and Workflow

In [ ]:
# =============================================================================
# HYPERPARAMETERS (from config, can be overridden)
# =============================================================================
# Use config defaults or override for experimentation

net_config = config.network
train_config = config.training

# Print current configuration
print("Network configuration:")
print(f"  DeepSet: dim={net_config.summary_dim}, depth={net_config.deepset_depth}, " +
      f"width={net_config.deepset_width}, dropout={net_config.deepset_dropout}")
print(f"  Flow: depth={net_config.flow_depth}, hidden={net_config.flow_hidden}, " +
      f"dropout={net_config.flow_dropout}")
print(f"\nTraining configuration:")
print(f"  lr={train_config.initial_lr}, batch={train_config.batch_size}, " +
      f"epochs={train_config.epochs}")

In [ ]:
# Build networks using factory function
summary_network, inference_network = build_networks(net_config)

print(f"Networks created:")
print(f"  DeepSet: summary_dim={net_config.summary_dim}, depth={net_config.deepset_depth}")
print(f"  CouplingFlow: depth={net_config.flow_depth}, hidden={net_config.flow_hidden}")

In [ ]:
# Optimizer with decaying learning rate
steps_per_epoch = train_config.batch_size * train_config.batches_per_epoch
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=train_config.initial_lr,
    decay_steps=steps_per_epoch,
    decay_rate=train_config.decay_rate,  
    staircase=True,
)
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

# Create workflow
workflow = bf.BasicWorkflow(
    simulator=simulator,
    adapter=adapter,
    inference_network=inference_network,
    summary_network=summary_network,
    optimizer=optimizer,
    inference_conditions=["N", "p_alloc", "prior_df", "prior_scale"],
    checkpoint_name="ancova_cont_2arms"
)

print("Workflow created (model will be built on first forward pass)")

## Training

Train the approximator to learn the posterior distribution of `b_covariate`.

In [ ]:
# MovingAverageEarlyStopping imported from utils (see imports cell)
print("MovingAverageEarlyStopping available from utils")

In [ ]:
callback = MovingAverageEarlyStopping(
    window=train_config.early_stopping_window, 
    patience=train_config.early_stopping_patience
)

history = workflow.fit_online(
    epochs=train_config.epochs, 
    batch_size=train_config.batch_size,     
    num_batches_per_epoch=train_config.batches_per_epoch, 
    validation_data=train_config.validation_sims,
    callbacks=[callback]
)

In [ ]:
f = bf.diagnostics.plots.loss(history)

## Diagnostics

Validate the trained approximator with simulation-based calibration and recovery diagnostics.

### All-in-One Diagnostic Plots

Generate all standard diagnostic plots at once using the built-in method.

In [ ]:
# Set the number of posterior draws you want to get
num_samples = 1000

# Simulate validation data (unseen during training)
val_sims = simulator.sample(1000)

# Obtain num_samples samples of the parameter posterior for every validation dataset
post_draws = workflow.sample(conditions=val_sims, num_samples=num_samples)

# Compute metrics for the validation set
metrics = workflow.compute_default_diagnostics(test_data=val_sims)
metrics["b_group"]

In [ ]:
print(val_sims.keys())
print(val_sims['N'])
print(val_sims['p_alloc'])
print(val_sims['group'].dtype)

In [ ]:
# Import plot functions from the package
from rctbp_bf_training.plotting import diagnostics as plot
importlib.reload(plot)

### Combined Diagnostic Dashboard

All calibration diagnostics in one comprehensive view.

In [ ]:
# Create diagnostic dashboard using BayesFlow plots + coverage difference
fig = plot.plot_diagnostic_dashboard(
    estimates=post_draws,
    targets=val_sims,
    param_key="b_group",
    variable_name=r"$b_2$ (treatment effect)"
)
plt.show()

# Validation on Conditions Grid

### Simulation Function
Function to simulate and fit one condition on a grid of parameters.

In [ ]:
# Import generic validation functions from the package
from rctbp_bf_training.core import validation as functions_validation
importlib.reload(functions_validation)

from rctbp_bf_training.core.validation import (
    run_validation_pipeline,
    extract_calibration_metrics,
    make_bayesflow_infer_fn,
)

# simulate_cond_batch and make_simulate_fn imported from rctbp_bf_training.models.ancova.model
print("Validation functions loaded from:")
print("  - rctbp_bf_training.core.validation: run_validation_pipeline, make_bayesflow_infer_fn")
print("  - rctbp_bf_training.models.ancova.model: simulate_cond_batch, make_simulate_fn")

### Condition Grid

In [ ]:
# Define parameter ranges for the validation grid
b_covariate_vals = [-0.5, 0.5]
b_group_vals = [0.0, 0.5, 1.0]
N_vals = [20, 1000]
p_alloc_vals = [0.5]
prior_df_vals = [0, 3]  # 0 = Normal, 10 = moderate t, 50 = nearly Normal
prior_scale_vals = [0.1, 1, 10]

# Build conditions grid with proper format for run_validation_pipeline
conditions = []
for idx, (n, p, b_cov, b_grp, pdf, psc) in enumerate(
    product(N_vals, p_alloc_vals, b_covariate_vals, b_group_vals, prior_df_vals, prior_scale_vals)
):
    conditions.append({
        "id_cond": idx,
        "n_total": n,
        "p_alloc": p,
        "b_covariate": b_cov,
        "b_arm_treat": b_grp,  # true treatment effect
        "prior_df": pdf,
        "prior_scale": psc,
    })

print(f"Total conditions: {len(conditions)}")
print(f"Example condition: {conditions[0]}")

### Run Validation

In [ ]:
# Use the trained workflow's approximator (or load from checkpoint)
# model_path = "checkpoints/ancova_cont_2arms.keras"
# model_ancova = keras.saving.load_model(model_path)
model_ancova = workflow.approximator

# Create adapters using imported factory functions
simulate_fn = make_simulate_fn(rng=RNG)
infer_fn = make_bayesflow_infer_fn(
    model_ancova, 
    param_key="b_group",
    data_keys=["outcome", "covariate", "group"],
    context_keys={"N": int, "p_alloc": float, "prior_df": float, "prior_scale": float}
)

# Run the complete validation pipeline
results = run_validation_pipeline(
    conditions_list=conditions,
    n_sims=1000,
    n_post_draws=1000,
    simulate_fn=simulate_fn,
    infer_fn=infer_fn,
    true_param_key="b_arm_treat",
    verbose=True
)

In [ ]:
# Extract results
metrics = results["metrics"]
metrics["condition_metrics"]


In [ ]:
# Extract results
metrics = results["metrics"]

# Condition-level summary statistics
display(metrics["condition_summary"].round(3))

In [ ]:
# Plot comprehensive SBC diagnostics (4 panels: histogram, ECDF, coverage, recovery)
from rctbp_bf_training.plotting import diagnostics as plot
importlib.reload(plot)

# Works directly on the metrics dict - shows all conditions combined
fig = plot.plot_sbc_diagnostics(metrics)

# Print key SBC test statistics
print(f"SBC KS p-value: {metrics['summary']['sbc_ks_pvalue']:.4f}")
print(f"SBC C2ST accuracy: {metrics['summary']['sbc_c2st_accuracy']:.4f} (0.5 = well-calibrated)")

In [ ]:
import functions_plots as plot
importlib.reload(plot)
max_conditions=len(metrics['condition_metrics'])
fig = plot.plot_ecdf_by_condition(metrics, max_conditions=max_conditions)

In [ ]:
importlib.reload(plot)
max_conditions=len(metrics['condition_metrics'])
fig = plot.plot_histogram_by_condition(metrics, max_conditions=max_conditions)

In [ ]:
importlib.reload(plot)
max_conditions=len(metrics['condition_metrics'])
fig = plot.plot_coverage_by_condition(metrics, max_conditions=max_conditions)

## Saving the Trained Model

In [ ]:
# Save the trained model with metadata
metadata = get_model_metadata(config)
save_path = Path("checkpoints") / "ancova_cont_2arms"
saved_path = save_model_with_metadata(workflow.approximator, save_path, metadata)

print(f"Model saved to: {saved_path}")
print(f"Metadata saved to: {saved_path.with_suffix('.json')}")

In [ ]:
# Load model with metadata
model_path = "checkpoints/ancova_cont_2arms"
loaded_model, loaded_metadata = load_model_with_metadata(model_path)

print("Model loaded successfully")
if loaded_metadata:
    print(f"Created: {loaded_metadata.get('created_at', 'unknown')}")
    print(f"Model type: {loaded_metadata.get('model_type', 'unknown')}")